# 跨工作表運算

我們在上一堂課利用 Python 與 Excel 這兩個工具打造出了一個回測系統，此工具允許我們能夠將回測流程自動化。但是，講到實用性，一般我們在投資股票時，不會只投資一支個股，而是多支股票，或者可以說由多個不同的個股組成的一個投資組合。

現在假設我們很費力的將多個個股的歷史資料整理在不同的工作簿内，而我們希望上一課的回測系統能夠對所有的股票進行回測，接下來我們就來看要如何能夠透過程式化的方法來做到這件事。

要如何能夠讓我們的回測系統程式能夠自行將所有股票的回測都執行完畢呢？這個時候我們就需要學會如何做跨工作表的運算，也就是如何透過程式操作多個工作表與工作簿的方法。


# xlwings 的工作表集合

假設今天我們搜集了四個不同股票的歷史資料，這四個股票分別是臺積電、玉山、鴻海、台灣50，這四支股票的歷史股價分別被整理在不同的工作簿内：

![](https://drive.google.com/uc?export=download&id=1qydsOfLS7GtNYD6rH-rCXJmOTGm-ltaF)

而每一個工作簿内股票的歷史資料格式都相同：

![](https://drive.google.com/uc?export=download&id=1rdyGuyW5N4UMYjzcufgcbeYQOpTFkXB2)


In [ ]:
import xlwings as xw
# 建立新的工作簿
wb = xw.Book()

接下來我們可以透過以下程式碼來檢查新工作簿内的工作表：

In [ ]:
wb.sheets

這個新建立的工作表上只有一個名爲 `工作表1` 的工作表，今天我們若要選擇該工作表，可以透過以下兩種方式：

In [ ]:
wb.sheets[0]

In [ ]:
wb.sheets["工作表1"]

In [ ]:
sheet = wb.sheets["工作表1"]

In [ ]:
print(sheet)

# 新增工作表

由於我們要匯整的資料很多，因此我們希望程式在讀取到一個新的個股資料時，能夠自動建立一個新的工作表，接下來再將資料寫入該工作表，我們可以透過以下程式碼新增一個工作表：

![](https://drive.google.com/uc?export=download&id=1w-k_wNMEj57Ci4EUDrNt3rPX16y2tIJP)

In [ ]:
new_sheet = wb.sheets.add()

In [ ]:
print(new_sheet)

In [ ]:
wb.sheets

In [ ]:
new_sheet = wb.sheets.add(name="2330")
print(new_sheet)

![](https://drive.google.com/uc?export=download&id=1r-TMlIjhY8A_UsZ3n5_9xDmmoz-F21ff)

In [ ]:
# 開啓 2330 歷史股價工作簿
wb_2330 = xw.Book("2330_history.xlsx")
# 建立新的工作表
sheet_2330 = wb.sheets.add(name="2330")
# 將歷史股價資料寫入新的工作表
sheet_2330.range("A1:H249").value = wb_2330.sheets["2330_history"].range("A1:H249").value

# 動態偵測資料筆數

In [ ]:
# 請確認所有的 Excel 檔案和此 Notebook 檔案都是在同一個資料夾底下
wb_2330 = xw.Book("2330_history.xlsx")
wb_2317 = xw.Book("2317_history.xlsx")
wb_2884 = xw.Book("2884_history.xlsx")
wb_0050 = xw.Book("0050_history.xlsx")

In [ ]:
wb_2330.sheets[0]

In [ ]:
sheet = wb_2330.sheets["2330_history"]
print(sheet)

在我們開始複製資料之前，我們先觀察一下所有的工作簿。

如果讀者仔細的觀察一下工作表，就會發現，今天每一支股票的歷史資料筆數其實是不同的，舉例來説，我們的臺積電（2330）工作簿的資料一共有 249 筆資料，但是玉山銀行（2884）則只有 173 筆資料。

# range.end 偵測範圍

問題來了，要是試算表上**多了或少了一筆資料，原本的程式就爆炸了...**

**若你寫出了一個需要頻繁修改的程式，那就失去了自動化的意義**

我們需要一個能夠動態偵測我們有幾筆資料需要計算的方法

在 VBA 的世界裡，有個好用的函數叫做 **End()**，可以動態偵測一從一個儲存格開始，**連續、有值範圍的最後一個儲存格**：


|  **常數**       | **方向**          |
| ------------- |:-----------:| 
| **End(xlDown)**    | ↓      | 
| **End(xlUp)**     | ↑      | 
| **End(xlRight)** | →      |
| **End(xlRight)** | ←      |




範例1：會從 A1 儲存格開始，**往下**查找到最後一個有值的儲存格
```vb
last_row = Range("A1").End(xlDown).Row
```

範例2：會從 A1 儲存格開始，**往右**查找到最後一個有值的儲存格
```vb
last_col = Range("A1").End(xlToRight).Column
```

幸運的是，`xlwings` 也具備了一模一樣的功能：

In [ ]:
from xlwings.constants import Direction
# 從 A1 儲存格開始，往下查找到最後一個有值的儲存格
sheet = wb_2330.sheets["2330_history"]
last_cell = sheet.range("A1").end(Direction.xlDown)
print(last_cell)

In [ ]:
# 把從 A1 開始，最右下角的儲存格底色換成紅色
last_cell.color = (255, 0, 0)

In [ ]:
from xlwings.constants import Direction
# 從 A1 儲存格開始，往下查找到最後一個有值的儲存格
last_cell = sheet.range("D4").end(Direction.xlDown)
print(last_cell)
# 會找到 D249
last_cell = sheet.range("D4").end(Direction.xlUp)
print(last_cell)
# 會找到 D1
last_cell = sheet.range("D4").end(Direction.xlToLeft)
print(last_cell)
# 會找到 A4
last_cell = sheet.range("D4").end(Direction.xlToRight)
print(last_cell)
# 會找到 H4

In [ ]:
# 另一種比較簡潔的寫法則是...
last_cell = sheet.range("D4").end("down")
print(last_cell)
# 會找到 D249
last_cell = sheet.range("D4").end("up")
print(last_cell)
# 會找到 D1
last_cell = sheet.range("D4").end("left")
print(last_cell)
# 會找到 A4
last_cell = sheet.range("D4").end("right")
print(last_cell)
# 會找到 H4

![](https://drive.google.com/uc?export=download&id=185_o3ZLPFvN_CE_aTR-ih2_WMWfGVvTj)

In [ ]:
last_cell = sheet.range("A1").end("down")
last_cell.row

![](https://drive.google.com/uc?export=download&id=16tOMtw6kqmufe2mCna4cw-ygOglXHsVS)

In [ ]:
dates = sheet.range(f"A1:A{last_cell.row}").value
prices = sheet.range(f"E1:E{last_cell.row}").value

dates, prices

In [ ]:
new_sheet.range("A1").value = dates

結果有些出乎我們意料：

![](https://drive.google.com/uc?export=download&id=1_89rfTmQWmTaGY5-RM2F30LDRPEM6TSc)

原因在於當我們把 `A1:A249` 這個範圍讀取出來時，由於該範圍的大小是 **249 x 1**，因此 xlwings 在預設上是將其變成一個一維陣列，大小是 **1 x 249**，因此造成了上述現象。

要修正這個問題，我們可以使用 range 方法的 options 屬性，將關鍵字引數 `ndim` 設定成 `2`：

In [ ]:
dates = sheet.range(f"A1:A{last_cell.row}").options(ndim=2).value
dates

In [ ]:
prices = sheet.range(f"E1:E{last_cell.row}").options(ndim=2).value
prices

最後再寫入新工作表的 A 欄與 B 欄：

In [ ]:
new_sheet.range("A1").value = dates
new_sheet.range("B1").value = prices

結果：

![](https://drive.google.com/uc?export=download&id=1mvn8RlvtQE5vNpGnaDhyTdHy630eHkQK)



接下來我們設定好新工作表的表頭：

![](https://drive.google.com/uc?export=download&id=1moAiqTKEeltT7wv1CtxMVDdLv0ie1t5j)

In [ ]:
headers = ["日期", "收盤價", "三日移動平均", "買入股數", "賣出股數", "持有股數","持有現金", "總資產", "起始資金"]
new_sheet.range("A1").value = headers

我們還需要設定 I 欄的格式：

![](https://drive.google.com/uc?export=download&id=1CzzZbe-V8-eKblvtdRm7lT55TAgHJxas)

In [ ]:
new_sheet.range("I2").value = "交易股數"
new_sheet.range("I3").value = "總收益"

# 將多個工作簿的資料匯整到同一個檔案

![](https://drive.google.com/uc?export=download&id=1U8upbX73cWcbQV1CGb-uqZReInqb0lt5)


In [ ]:
stocks = ["2330", "2884", "2317", "0050"]
headers = ["日期", "收盤價", "三日移動平均", "買入股數", "賣出股數", "持有股數","持有現金", "總資產", "起始資金"]

new_wb = xw.Book()

for stock in stocks:
    wb = xw.Book(f"{stock}_history.xlsx")
    sheet = wb.sheets[0]
    last_row = sheet.range("A1").end("down").row
    new_sheet = new_wb.sheets.add(name=stock)
    new_sheet.range("A1").value = sheet.range(f"A1:A{last_row}").options(ndim=2).value
    new_sheet.range("B1").value = sheet.range(f"E1:E{last_row}").options(ndim=2).value
    new_sheet.range("A1").value = headers
    new_sheet.range("I2").value = "交易股數"
    new_sheet.range("I3").value = "總收益"

new_wb.save("投資組合歷史資料.xlsx")

# 投資組合回測

請確保：

1. 每一個個股工作表都有設定**起始資金**以及**交易股數**
2. 將**工作表1**刪除

![](https://drive.google.com/uc?export=download&id=1ctplmK_np_R6hfLPNcerpc8m8X0gBeXl)

In [ ]:
import xlwings as xw

wb = xw.Book(r"在這裏輸入你放置 投資組合歷史資料.xlsx 檔案在你的電腦的路徑，包含副檔名 .xlsx")
sheet = wb.sheets["2330"]

## 計算三日移動平均

In [ ]:
last_row = sheet.range("A1").end("down").row

for i in range(4, last_row+1):
    formula = f"=Average(B{i-2}:B{i})"
    sheet.range(f"C{i}").formula = formula

In [ ]:
def calculate_sma_3d(sheet):
    last_row = sheet.range("A1").end("down").row

    for i in range(4, last_row+1):
        formula = f"=Average(B{i-2}:B{i})"
        sheet.range(f"C{i}").formula = formula

## 計算第一天的交易資料

In [ ]:
# 截取當天收盤價
price = sheet.range(f"B4").value
# 截取當天的 3日移動平均
sma3d = sheet.range(f"C4").value
# 從 J2 截取我們的交易股數，也就是 1000
shares = sheet.range("J2").value
# 手動設定第四行，也就是第一天的資料
if price > sma3d:
    sheet.range(f"D4").value = shares
else:
    # 若上述條件不符和，就買入 0 股，（在 E 欄顯示 0）
    sheet.range(f"D4").value = 0
# 若 3日 > ，而且昨天的持有股數大於 1000 股，就賣出 1000 股
if price < sma3d:
    sheet.range(f"E4").value = shares
else:
    sheet.range(f"E4").value = 0

sheet.range("F4").value = sheet.range("D4").value - sheet.range("E4").value
sheet.range("G4").value = sheet.range("J1").value - sheet.range("F4").value * sheet.range("B4").value
sheet.range("H4").value = sheet.range("G4").value + sheet.range("F4").value * sheet.range("B4").value

In [ ]:
def calculate_first_day(sheet):
    # 截取當天收盤價
    price = sheet.range(f"B4").value
    # 截取當天的 3日移動平均
    sma3d = sheet.range(f"C4").value
    # 從 J2 截取我們的交易股數，也就是 1000
    shares = sheet.range("J2").value
    # 手動設定第四行，也就是第一天的資料
    if price > sma3d:
        sheet.range(f"D4").value = shares
    else:
        # 若上述條件不符和，就買入 0 股，（在 E 欄顯示 0）
        sheet.range(f"D4").value = 0
    # 若 3日 > ，而且昨天的持有股數大於 1000 股，就賣出 1000 股
    if price < sma3d:
        sheet.range(f"E4").value = shares
    else:
        sheet.range(f"E4").value = 0

    sheet.range("F4").value = sheet.range("D4").value - sheet.range("E4").value
    sheet.range("G4").value = sheet.range("J1").value - sheet.range("F4").value * sheet.range("B4").value
    sheet.range("H4").value = sheet.range("G4").value + sheet.range("F4").value * sheet.range("B4").value

## 執行回測

In [ ]:
sheet.range("J1").name = "起始資金"
sheet.range("J2").name = "交易股數"

# 從 J2 截取我們的交易股數，也就是 1000
shares = sheet.range("J2").value
# 實作交易策略
for i in range(5, last_row+1):
    # D 欄公式
    sheet.range(f"D{i}").formula = f"=IF(AND(B{i}>C{i}, B{i}*交易股數<=G{i-1}), {shares}, 0)"
    # E 欄公式
    sheet.range(f"E{i}").formula = f"=IF(AND(B{i}<C{i}, F{i-1}>=交易股數), {shares}, 0)"
    # 持有股數，算法是前一天的持有股數 + 今天的買入股數 - 今天的賣出股數
    sheet.range(f"F{i}").formula = f"=D{i}-E{i}+F{i-1}"
    # 持有資金，算法是前一天的持有資金 + 今日收盤價 x (今天的賣出股數 - 今天的買入股數)
    sheet.range(f"G{i}").formula = f"=(E{i}-D{i})*B{i}+G{i-1}"
    # 總資產則是持有股數 x 今日收盤價 + 今日持有資金
    sheet.range(f"H{i}").formula = f"=F{i}*B{i}+G{i}"

# 計算并且將總收益顯示在 J3
sheet.range("J3").value = sheet.range(f"H{last_row}").value - sheet.range(f"J1").value

In [ ]:
def sma_back_test(sheet):
    sheet.range("J1").name = "起始資金"
    sheet.range("J2").name = "交易股數"
    # 從 J2 截取我們的交易股數，也就是 1000
    shares = sheet.range("J2").value
    last_row = sheet.range("A1").end("down").row
    # 實作交易策略
    for i in range(5, last_row+1):
        # D 欄公式
        sheet.range(f"D{i}").formula = f"=IF(AND(B{i}>C{i}, B{i}*交易股數<=G{i-1}), {shares}, 0)"
        # E 欄公式
        sheet.range(f"E{i}").formula = f"=IF(AND(B{i}<C{i}, F{i-1}>=交易股數), {shares}, 0)"
        # 持有股數，算法是前一天的持有股數 + 今天的買入股數 - 今天的賣出股數
        sheet.range(f"F{i}").formula = f"=D{i}-E{i}+F{i-1}"
        # 持有資金，算法是前一天的持有資金 + 今日收盤價 x (今天的賣出股數 - 今天的買入股數)
        sheet.range(f"G{i}").formula = f"=(E{i}-D{i})*B{i}+G{i-1}"
        # 總資產則是持有股數 x 今日收盤價 + 今日持有資金
        sheet.range(f"H{i}").formula = f"=F{i}*B{i}+G{i}"

    # 計算并且將總收益顯示在 J3
    sheet.range("J3").value = sheet.range(f"H{last_row}").value - sheet.range(f"J1").value

In [ ]:
# 迭代每一個工作表
for sheet in wb.sheets:
    # 開啓工作表
    sheet.activate()
    # 計算三日移動平均
    calculate_sma_3d(sheet)
    # 計算第一天的報酬率
    calculate_first_day(sheet)
    # 執行回測
    sma_back_test(sheet)

# 呈現回測結果

在成功把所有股票的回測都跑完之後，剩下的事就是實際檢驗到底我們的投資策略賺了多少錢，雖然現在我們的程式在跑完後，就會把該試算表的總收益寫入 K15 儲存格，但是要把每一個試算表上的總收益加起來，還是很麻煩，那還不如就直接透過程式自動把所有的總收益加起來，算出整個投資組合的總收益。

![](https://drive.google.com/uc?export=download&id=13ZLtwB8BvImq_70X_pol3tkQR0_aXT9L)

In [ ]:
profit = 0

for sheet in wb.sheets:
    if sheet.name != "投資組合報表":
        # 開啓工作表
        sheet.activate()
        # 計算三日移動平均
        calculate_sma_3d(sheet)
        # 計算第一天的報酬率
        calculate_first_day(sheet)
        # 執行回測
        sma_back_test(sheet)
        # 執行完回測，讀取該個股總收益
        profit += sheet.range("J3").value

# 顯示加總後、也就是投資組合的收益
print(f"投資組合收益： ${profit}")

In [ ]:
sheet.activate()

import time

for sheet in wb.sheets:
    sheet.activate()
    time.sleep(2)

當然，光是用 print 函數將我們的投資組合收益呈現在 Jupyter Notebook 上很不直覺，因此我們可以將結果呈現在工作表上，接下來我們可以建立一個新的工作表：

In [ ]:
portfolio = wb.sheets.add(name="投資組合報表")
portfolio.range("A1").value = "個股名稱"
portfolio.range("B1").value = "個股收益金額（$）"
portfolio.range("C1").value = "個股收益（%）"
portfolio.range("A11").value = "總收益金額（$）"
portfolio.range("C11").value = "總收益率（%）"

![](https://drive.google.com/uc?export=download&id=16ncM0bKpQLygNvte5XPwLsnb-1Z7H_5z)

In [ ]:
# 收益
profit = 0
# 成本
cost = 0
# 起始 row 值
row = 2

portfolio = wb.sheets["投資組合報表"]

for sheet in wb.sheets:
    # 判斷目前迭代到的工作表是否為投資組合報告
    if sheet.name == "投資組合報表":
        # 若是，就不執行回測
        continue
    sheet.activate()
    calculate_sma_3d(sheet)
    calculate_first_day(sheet)
    sma_back_test(sheet)
    # 將目前工作表名稱寫入投資組合報告 A 欄
    portfolio.range(f"A{row}").value = sheet.name
    # 將目前工作表的收益金額寫入投資組合報告 B 欄
    portfolio.range(f"B{row}").value = sheet.range("J3").value
    # 將目前工作表的收益比例寫入投資組合報告 B 欄
    portfolio.range(f"C{row}").value = sheet.range("J3").value / sheet.range("J1").value
    # 更新 row 值
    row += 1
    # 叠加總成本金額
    cost += sheet.range("J1").value
    # 叠加總收益金額
    profit += sheet.range("J3").value

portfolio.range("B11").value = profit
portfolio.range("D11").value = profit / cost

最後稍微調整一下報告工作表的格式，就大功告成了！

![](https://drive.google.com/uc?export=download&id=13ZLtwB8BvImq_70X_pol3tkQR0_aXT9L)

# 功課

請嘗試用 Python 寫一個程式：

1. 將**投資組合歷史資料.xlsx** 檔案内不同個股的資料分別存入四個不同的 Excel 工作簿内
2. 格式與這節課一開始的四個歷史資料 Excel 工作簿一樣


![](https://drive.google.com/uc?export=download&id=1qydsOfLS7GtNYD6rH-rCXJmOTGm-ltaF)
